# --- Day 10: Factory ---
https://adventofcode.com/2025/day/10

In [11]:
import re

def getInput():
	with open("schematics.txt") as file:
		return file.read()
	
def getSampleInput():
	with open("sampleInput.txt") as file:
		return file.read()

In [40]:
lightsAndButtons = []
for line in getInput().splitlines():
	# Get the lights (the characters in the brackets)
	indicatorLightDiagram = re.findall(r'\[([.#]+)\]', line)[0]

	# Get and format buttons
	buttons = [x[0].strip("()") for x in re.finditer(r'\(((\d,?)+)\)', line)]
	buttons = [list(map(int, x.split(","))) for x in buttons]
	lightsAndButtons.append((indicatorLightDiagram, buttons))


def toggleLights(currentLights: str, button: list[int]) -> str:
	"""Returns lights with all indexes in button toggled"""
	newLights = ""
	for i, light in enumerate(currentLights):
		if i in button:
			newLights += "." if light == "#" else "#"
		else:
			newLights += light
	
	return newLights

def getMinButtonPresses(goal: str, buttons: list[list[int]]) -> int:
	"""Uses bfs to get the minimum number of button presses to get to the goal"""
	queue = [["." * len(goal), 0]]
	explored = set()
	
	# Loop until the queue is empty
	while queue:
		# Get the next light layout in the queue
		# If we've hit the goal then return the number of steps it took to get there
		currentLights, steps = queue.pop(0)

		# Loop through all lights after each button press
		for successor in [toggleLights(currentLights, button) for button in buttons]:
			if successor == goal:
				return steps + 1
			if successor not in explored:
				queue.append([successor, steps + 1])
				explored.add(successor)

totalButtonPresses = 0
for lights, buttons in lightsAndButtons:
	totalButtonPresses += getMinButtonPresses(lights, buttons)

print(f"Total number of button presses: {totalButtonPresses}")

Total number of button presses: 447
